In [9]:
import import_ipynb
import numpy as np
import math
import matplotlib.pyplot as plt
import gates as gate
import register as reg
plt.style.use('dark_background')

importing Jupyter notebook from gates.ipynb
importing Jupyter notebook from register.ipynb


In [31]:
# This is the entire Grover's algorithm.
def grover(qubits=4,key=0,track=True,wordsearch=False): 
    """
    Apply Grover's Algorithm to a Quantum Register
    

    Grover's algorithm here runs in a few simple steps. Each section has been documented to
    describe its impact, but the whole process can be qualitatively summarized as follows:
    1. Create a quantum register with "qubits" number of qubits.
    2. Force this register into superposition.
    3. Run Grover's Iteration O(sqrt(N)) times (more specifically calculated below):
        - Apply oracle, which flips the sign of target basis
        - Apply Diffuser: Hadmard,2|s><s|-I, Hadamard.
    4. Collapse the system by measuring its amplitudes.
    
            Parameters:
                qubits: int
                    # of qubits to use
                key: int
                    basis state to search for
                track: Boolean
                    States whether to store steps of Grover's or just end product
                wordsearch:
                    States whether to prompt for inputs and search with them or not

            Returns:
                end: double array
                    amplitudes measured at the end of the circuit
                arr: double array
                    amplitude change over time (ONLY IF TRACK=TRUE)
                span: int
                    basis size (ONLY IF WORDSEARCH=TRUE)
                places: int array
                    key values defined by prompt (ONLY IF WORDSEARCH=TRUE)


    """
    # define basis space (span), how many iterations Grover should run (grover_limit)
    span = 1<<qubits  
    grover_limit = int(np.pi*np.sqrt(span)/4.0) 
    
    # If Wordsearch=True, redefine paramaters in terms of inputs received.
    # -------------------------------
    if wordsearch==True: 
        key, span = wordsalad()
        qubits = 1
        while 2**qubits<span:
            qubits+=1
        span = 2**qubits
        grover_limit = len(key)*int(np.pi*np.sqrt(span)/4.0) 
    #--------------------------------     
        
    # Create list of initial gates and registers: initial register in |000...0> state (initial), oracle matrix (apply_oracle),
    # diffusion gate (apply_diffusion), harmard matrix for the entire system (Hinitial), superposition (global hadmard applied
    # to the initial 0 state system)
    p =[]
    initial = reg.given(0,span) 
    apply_oracle = gate.oracle(span,key) 
    apply_diffusion = gate.diffusion(qubits)
    Hinitial = gate.globalapp(qubits, gate.hgate()) 
    superposition = reg.register(np.dot(initial.var, Hinitial.matrix)) 
    
    # Grover's Iterations as a product of the oracle gate (apply_oracle) and diffusion gate (apply_diffusion), run grover_limit
    # times. Once this is done, apply to superposition values. If track=True, the same is done, but this time each adjustment
    # to Grover's Iteration is recorded so that it might be visualised later.
    if track==False:
        Grovers_Iteration = apply_oracle @ apply_diffusion 
        for i in range(grover_limit):
            step = apply_oracle @ apply_diffusion 
            Grovers_Iteration = Grovers_Iteration @ step 
        circuit_end = reg.register(np.dot(superposition.var,Grovers_Iteration))
        end = circuit_end 
    else:
        p.append(superposition.var)
        for i in range(grover_limit):
            superposition=reg.register(np.dot(superposition.var,apply_oracle))
            p.append(superposition.var)
            superposition=reg.register(np.dot(superposition.var,apply_diffusion))
            p.append(superposition.var)
        end = superposition
        arr = np.array(p)
        
    # If Wordsearch=True, return the end amplitudes, the highest of which relate to
    # the definied keys (places). Also return the basis size (span), since that was
    # created, instead of defined, based on the length of the sentence provided. 
    # -------------------------------
    if wordsearch==True:
        search = np.copy(end.var**2) 
        places = [] 
        while len(places)<len(key): 
            top = max(search)
            for i in range(len(search)):
                if search[i]==top:
                    places.append(i)
                    search[i]=0
        places = sorted(places)    
        print("Position(s): " + str(places))
        if track==False:
            return end,places,span
        else:
            return end,arr,places,span    
    #------------------------------------
    
    # Final returns as described in initial docustring.
    if track==False:
        return end
    else:
        return end,arr

In [25]:
def wordsalad():
    """
    Prompts user input (sentence and word to look for)
        
        returns:
            span: int
                basis size (depends on sentence inputted)
            key: int array
                position(s) of word in string to search for
    
    """
    salad = input("Enter text: ")
    salad=salad.split()
    span = len(salad)
    word = input("What word do you want to search for? ")
    key = []
    for i in range(span):
        if salad[i]==word:
            key.append(i)
    return key, span